# Array Benchmarks

Benchmark array operations for MBO data processing.

In [ ]:
import json
import timeit
from datetime import datetime
from pathlib import Path

import numpy as np

import mbo_utilities as mbo
from mbo_utilities.widgets import select_file
from mbo_utilities.phasecorr import apply_scan_phase_offsets, _phase_corr_2d
from fastplotlib.utils import subsample_array

In [ ]:
fpath = select_file(title="Select image file")
print(f"Selected: {fpath}")

In [ ]:
arr = mbo.imread(fpath)
print(f"Type: {type(arr).__name__}")
print(f"Shape: {arr.shape}")
print(f"Dtype: {arr.dtype}")
print(f"Elements: {np.prod(arr.shape):,}")

In [ ]:
results = {
    "file": str(fpath),
    "array_type": type(arr).__name__,
    "shape": arr.shape,
    "dtype": str(arr.dtype),
    "timestamp": datetime.now().isoformat(),
    "benchmarks": {}
}

mid_idx = arr.shape[0] // 2 if arr.ndim >= 3 else 0
ignore_dims = tuple(range(arr.ndim))[-2:]

## Single Frame Access

In [ ]:
%%timeit -o -q
frame = np.asarray(arr[mid_idx])

In [ ]:
results["benchmarks"]["single_frame_access"] = {"mean": _.average, "std": _.stdev}
print(f"Single frame access: {_.average*1000:.2f} ± {_.stdev*1000:.2f} ms")

## Histogram: Single Frame vs Full Array

In [ ]:
%%timeit -o -q
frame = np.asarray(arr[mid_idx])
np.histogram(frame.ravel(), bins=100)

In [ ]:
results["benchmarks"]["histogram_single_frame"] = {"mean": _.average, "std": _.stdev}
print(f"Histogram (single frame): {_.average*1000:.2f} ± {_.stdev*1000:.2f} ms")

In [ ]:
%%timeit -o -q
sub = subsample_array(arr, ignore_dims=ignore_dims)
sub_real = sub[~(np.isnan(sub) | np.isinf(sub))]
np.histogram(sub_real, bins=100)

In [ ]:
results["benchmarks"]["histogram_full_array"] = {"mean": _.average, "std": _.stdev}
print(f"Histogram (full array): {_.average*1000:.2f} ± {_.stdev*1000:.2f} ms")

speedup = results["benchmarks"]["histogram_full_array"]["mean"] / results["benchmarks"]["histogram_single_frame"]["mean"]
print(f"Speedup: {speedup:.1f}x")

## Phase Correction

In [ ]:
frame = np.asarray(arr[mid_idx])
print(f"Frame shape: {frame.shape}")

In [ ]:
%%timeit -o -q
_phase_corr_2d(frame, upsample=5, border=3, max_offset=3, use_fft=False)

In [ ]:
results["benchmarks"]["phase_corr_compute"] = {"mean": _.average, "std": _.stdev}
print(f"Phase correlation (compute): {_.average*1000:.2f} ± {_.stdev*1000:.2f} ms")

In [ ]:
%%timeit -o -q
apply_scan_phase_offsets(frame, 1.5)

In [ ]:
results["benchmarks"]["phase_corr_apply"] = {"mean": _.average, "std": _.stdev}
print(f"Phase correction (apply): {_.average*1000:.2f} ± {_.stdev*1000:.2f} ms")

In [ ]:
%%timeit -o -q
_phase_corr_2d(frame, upsample=5, border=3, max_offset=3, use_fft=True)

In [ ]:
results["benchmarks"]["phase_corr_compute_fft"] = {"mean": _.average, "std": _.stdev}
print(f"Phase correlation (FFT): {_.average*1000:.2f} ± {_.stdev*1000:.2f} ms")

## Subsample Array

In [ ]:
%%timeit -o -q
subsample_array(arr, ignore_dims=ignore_dims)

In [ ]:
results["benchmarks"]["subsample_array"] = {"mean": _.average, "std": _.stdev}
print(f"Subsample array: {_.average*1000:.2f} ± {_.stdev*1000:.2f} ms")

sub = subsample_array(arr, ignore_dims=ignore_dims)
print(f"Subsampled shape: {sub.shape}, elements: {sub.size:,}")

## Summary

In [ ]:
print(f"\n{'='*50}")
print(f"Array: {type(arr).__name__} {arr.shape}")
print(f"{'='*50}")
for name, data in results["benchmarks"].items():
    print(f"{name:30s}: {data['mean']*1000:8.2f} ms")
print(f"{'='*50}")

In [ ]:
save_results = False  # Set True to save

if save_results:
    out_path = Path(fpath).parent / f"benchmark_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(out_path, "w") as f:
        json.dump(results, f, indent=2, default=str)
    print(f"Saved: {out_path}")